<a href="https://colab.research.google.com/github/staerkjoe/NLP_colab/blob/main/AML4NLP2025_E04_Intro_W%26B_Weave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction: Eploring Weights & Biases Weave

ITU KSAMLDS1KU - Advanced Machine Learning for Data Science 2025

by Jonathan Tiedchen, Eisuke Okuda, Stefan Heinrich,
& material by Bertram Højer, Kevin Murphy, and Chris Bishop.

All info and static material: https://learnit.itu.dk/course/view.php?id=3024752

-------------------------------------------------------------------------------

### 1. Install dependencies

In [ ]:
!pip install wandb weave transformers datasets torch

### 2. Imports

In [ ]:
import wandb
import weave
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

### 3. Initialize W&B and Weave

In [ ]:
# Login to W&B
wandb.login()

# Start a new run
wandb_run = wandb.init(
    project="llm_prompt_logging",
    config={
        "model": "sshleifer/tiny-gpt2",
        "max_new_tokens": 50,
        "temperature": 0.7
    }
)

# Initialize Weave client
client = weave.init("jonathantied-private/llm_prompt_logging")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


wandb: Initializing weave.


Output()

weave: Logged in as Weights & Biases user: jonathantied.
weave: View Weave data at https://wandb.ai/jonathantied-private/llm_prompt_logging/weave


Output()

weave: Logged in as Weights & Biases user: jonathantied.
weave: View Weave data at https://wandb.ai/jonathantied-private/llm_prompt_logging/weave


### 4. Load a Small Model (tiny GPT-2)

In [ ]:
model_name = wandb.config["model"]

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### 5. Define a Prompt Function with Logging

In [ ]:
def prompt_model(question, temperature=0.7, max_new_tokens=50):
    # Create Weave call
    call = client.create_call(
        op="prompt_model",
        inputs={
            "question": question,
            "temperature": temperature,
            "max_new_tokens": max_new_tokens
        }
    )

    # Encode input
    input_text = f"Q: {question}\nA:"
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = output_text.split("A:")[-1].strip()

    # Log to W&B
    wandb.log({"question": question, "answer": answer})

    # Finish Weave call
    client.finish_call(call, output={"answer": answer})

    return answer


### 6. Run a few examples

In [ ]:
questions = [
    "What is 2+2?",
    "Explain the capital of France.",
    "Who wrote the book 1984?"
]

for q in questions:
    ans = prompt_model(q, temperature=wandb.config["temperature"])
    print(f"Q: {q}\nA: {ans}\n")


Q: What is 2+2?
A: stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs

Q: Explain the capital of France.
A: stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs

Q: Who wrote the book 1984?
A: stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs s

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


weave: 🍩 https://wandb.ai/jonathantied-private/llm_prompt_logging/r/call/01991679-7442-7ff6-94f7-6aa4e959111e
weave: 🍩 https://wandb.ai/jonathantied-private/llm_prompt_logging/r/call/01991679-72a7-7a91-ba53-2296eebe4bca
weave: 🍩 https://wandb.ai/jonathantied-private/llm_prompt_logging/r/call/01991679-7421-796e-aa5f-3d86736111c6


### 7. Finish Run

In [ ]:
wandb.finish()


answer,stairs stairs stairs...
question,Who wrote the book 1...
